In [ ]:
from __future__ import print_function, division
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam ,RMSprop, SGD ,Adadelta ,Adagrad, Adamax
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

import numpy as np

class GAN():
    def __init__(self):
        self.rows = 1
        self.cols = 5825
        self.channels = 1
        self.qos_shape = (self.rows, self.cols)
        self.latent_dim = 100
        self.resultat = pd.DataFrame(columns = ['EPOCHS','RMSE_AUTOENCODERS'])

        optimizer = Adam(0.001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates vectors
        z = Input(shape=(self.latent_dim,))
        qos = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated vectors as input and determines validity
        validity = self.discriminator(qos)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.99))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.99))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.99))
        model.add(Dense(np.prod(self.qos_shape), activation='relu'))
        model.add(Reshape(self.qos_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        qos = model(noise)

        return model

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.qos_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        qos = Input(shape=self.qos_shape)
        validity = model(qos)

        return Model(qos, validity)

    def train(self, epochs, batch_size=273, sample_interval=50):

        # Load the dataset
        rtMatrix = pd.read_csv('./train.txt', delimiter="\t", header=None)
        
        # Rescale -1 to 1
        X_train = np.expand_dims(rtMatrix, axis=1)
        X_train = X_train / 20.
        

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of vectors
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            v_qos = X_train

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new vectors
            gen_v_qos = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(v_qos, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_v_qos, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            if epoch % sample_interval == 0:
                # three different tests

                #one=self.mean_all(epoch,d_loss[0], 100*d_loss[1], g_loss)
                #two=self.mean_every(epoch,d_loss[0], 100*d_loss[1], g_loss)
                three=self.test_autoencoder(epoch)
                self.resultat = self.resultat.append({'EPOCHS' : epoch, 'RMSE_AUTOENCODERS' : three}, ignore_index=True)
            if epoch == epochs-1:
                #Result 

                print(self.resultat)
                self.resultat.to_csv('resultat.txt', index=False,sep='\t',float_format='%.3f')
    def root_mean_squared_error(self,y_true, y_pred):
        K = tf.keras.backend
        return K.sqrt(K.mean(K.square(y_pred - y_true)))     
    def mean_all(self, epoch,d_loss, a_loss, g_loss):
        test = pd.read_csv('./test.txt', delimiter="\t", header=None)
        t_mean=test.mean()
        noise = np.random.normal(0, 1, (66, self.latent_dim))
        gen_qos = self.generator.predict(noise)
        # Rescale 
        gen_qos = 20 * gen_qos
        reduire=np.squeeze(gen_qos, axis=1)
        df = pd.DataFrame(data=reduire)
        mean = df.mean()
        mse = self.root_mean_squared_error(t_mean, mean)
        mse = mse.numpy()

        print(epoch ,' rmse entre la moyenne de tout les vecteurs original et les vecteurs generer ' ,mse,'' )

        return  mse

    def mean_every(self, epoch,d_loss, a_loss, g_loss):
        test = pd.read_csv('./test.txt', delimiter="\t", header=None) 
        t_mean=test.mean()
        mean_final=0

        #66 users

        for i in range(66):
            rt=test.iloc[i]
            fin=np.expand_dims(rt, axis=0)
            
            noise = np.random.normal(0, 1, (1, self.latent_dim))
            gen_qos = self.generator.predict(noise)
            # Rescale 
            gen_qos = 20 * gen_qos
            reduire=np.squeeze(gen_qos, axis=1)
            df = pd.DataFrame(data=reduire)
            mean = df.mean()

            mse = self.root_mean_squared_error(fin, reduire)
            mse = mse.numpy()
            mean_final = mean_final + mse
        result = mean_final/66

        print(epoch ,' la moyenne de tout les rmse entre chaque vectueur original et generer ' ,result,' ' )
        return result
    def test_autoencoder(self, epoch):
        mean_final=0
        def get_invalid_indices(vecteur):
            indice_1 = np.argwhere(vecteur > 19.).flatten()  # index of invalid values (no reponse after invocation)
            indice_2 = np.argwhere(vecteur <= 0.).flatten()  # index of invalid values (none invocation)
            indice_invalid_values_ds1 = np.append(indice_1, indice_2) # concatenate of all invalid valeus
            return indice_invalid_values_ds1
        rtMatrix = pd.read_csv('./rtMatrix.txt', delimiter="\t", header=None)
        rtMatrix = rtMatrix.drop(5825, axis=1)
        
        lay1= self.generator.layers[0].get_weights()[0].T
        lay3= self.generator.layers[3].get_weights()[0].T
        lay6= self.generator.layers[6].get_weights()[0].T
        lay9= self.generator.layers[9].get_weights()[0].T
        
        ba1= self.generator.layers[0].get_weights()[1]
        ba3= self.generator.layers[3].get_weights()[1]
        ba6= self.generator.layers[6].get_weights()[1]
        ba9= self.generator.layers[9].get_weights()[1]        
        
        autoencoder = Sequential()
        autoencoder.add(Dense(1024, input_shape=(5825,)))
        autoencoder.add(LeakyReLU(alpha=0.2))
        autoencoder.add(BatchNormalization(momentum=0.8))
        autoencoder.add(Dense(512))
        autoencoder.add(LeakyReLU(alpha=0.2))
        autoencoder.add(BatchNormalization(momentum=0.8))
        autoencoder.add(Dense(256))
        autoencoder.add(LeakyReLU(alpha=0.2))
        autoencoder.add(BatchNormalization(momentum=0.8))
        autoencoder.add(Dense(100,    activation='sigmoid', name="bottleneck"))
        
        K = tf.keras.backend
        K.set_value(autoencoder.layers[0].weights[0], lay9)
        K.set_value(autoencoder.layers[3].weights[0], lay6)
        K.set_value(autoencoder.layers[6].weights[0], lay3)
        K.set_value(autoencoder.layers[9].weights[0], lay1)

        K.set_value(autoencoder.layers[0].weights[1], ba6)
        K.set_value(autoencoder.layers[3].weights[1], ba3)
        K.set_value(autoencoder.layers[6].weights[1], ba1)
        #K.set_value(autoencoder.layers[9].weights[1], ba1)

        #339 users

        for i in range(339):
            indice_invalid_values = get_invalid_indices(rtMatrix.iloc[i,:].to_numpy())
            vecteur_user_ds1 = np.delete(rtMatrix.iloc[i,:].to_numpy(), indice_invalid_values, axis=None)
            vecteur_user_ds1 = np.expand_dims(vecteur_user_ds1, axis=0)
            rtMatrixt = np.expand_dims(rtMatrix.iloc[i], axis=0)

            train_x = rtMatrixt / 20

            encoded_data = autoencoder.predict(train_x)

            gen_qos = self.generator.predict(encoded_data)

            gen_qos = 20 * gen_qos
            reduire = np.delete(gen_qos, indice_invalid_values, axis=None)
            reduire= np.expand_dims(reduire, axis=0)

            mse = self.root_mean_squared_error(vecteur_user_ds1, reduire)
            mse = mse.numpy()

            mean_final = mean_final + mse
        result=mean_final/339
        print(epoch,' rmse with using the autoencoders ',result,' \n')
        return result

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=20000, batch_size=273, sample_interval=200)
